In [66]:
import os
import json
import pandas as pd
import numpy as np
import shutil
import random
from tqdm import tqdm
from pathlib import Path

# Functions

In [68]:
def get_val_to_col_name_dict(input_dir):
    '''
    Given an input directory containing the raw csv files return the `val_to_col_name_dict`
    '''
    val_to_col_name_dict = {}
    # Find the all Homographs and populate the `val_to_col_name_dict`
    for filename in tqdm(os.listdir(input_dir)):
        df = pd.read_csv(input_dir+filename, keep_default_na=False)
        column_names=list(df.columns)
        for idx, row in df.iterrows():
            for i in range(len(row.tolist())):
                if row[i] in val_to_col_name_dict:
                    val_to_col_name_dict[row[i]].add(column_names[i])
                else:
                    val_to_col_name_dict[row[i]] = set([column_names[i]])

    return val_to_col_name_dict

def get_homographs_and_clean_dataset(input_dir, output_dir, column_name_to_homograph_type, val_to_col_name_dict):
    '''
    Identifies all homographs and outputs a dataframe with them at `output_dir`.
    Also creates a new version on the input tables without he inclusion of the homographs and outputs them at `output_dir/no_homographs/`

    Returns the `homograph_info_df`
    '''
    Path(output_dir+'no_homographs/').mkdir(parents=True, exist_ok=True)
    homograph_info_dict={'value': [], 'filename': [], 'column_name' : [], 'type': [], 'subtype': [], "contents_row": []}

    for filename in tqdm(os.listdir(input_dir)):
        df = pd.read_csv(input_dir+filename, keep_default_na=False)
        column_names=list(df.columns)

        row_ids_with_homographs=set()
        for idx, row in df.iterrows():
            for i in range(len(row.tolist())):
                # Check if value is a homograph
                if row[i]=='':
                    row_ids_with_homographs.add(idx)
                elif len(val_to_col_name_dict[row[i]])>1:
                    row_ids_with_homographs.add(idx)
                    homograph_info_dict['value'].append(row[i])
                    homograph_info_dict['filename'].append(filename)
                    homograph_info_dict['column_name'].append(column_names[i])
                    homograph_info_dict['type'].append(column_name_to_homograph_type[column_names[i]]['type'])
                    homograph_info_dict['subtype'].append(column_name_to_homograph_type[column_names[i]]['subtype'])
                    homograph_info_dict['contents_row'].append(row.tolist())
            
        df_no_homographs = df.drop(labels=row_ids_with_homographs)
        df_no_homographs.to_csv(output_dir+'no_homographs/'+filename,index=False)

    homograph_info_df = pd.DataFrame.from_dict(homograph_info_dict)
    
    return homograph_info_df

def select_homographs_of_specified_type(homograph_df, num_homographs=10, type='traditional', subtype=None, seed=0):
    '''
    Returns a list of the selected homographs from the specified types. 

    Raises an error if it is not possible to extract the specified types
    '''

    # Check if the requested specification is possible (i.e., there are enough unique homographs)
    valid_tuples_df=homograph_df[homograph_df['type']==type]
    if subtype: valid_tuples_df=homograph_df[homograph_df['subtype']==subtype]
    if valid_tuples_df['value'].nunique() < num_homographs:
        raise ValueError('Not possible to extract ' + str(num_homographs) + ' homographs. There are only ' + str(valid_tuples_df['value'].nunique()) + ' available homographs with the specified parameters.')

    # Select the homographs
    np.random.seed(seed)
    selected_homographs=np.random.choice(valid_tuples_df['value'].unique(), size=num_homographs, replace=False)
    return selected_homographs

def construct_homograph_injected_dataset(input_dir, output_dir, homograph_info_df, num_homographs=10, type='traditional', subtype=None, seed=0, mode=None):
    '''
    Construct injected homograph datasets of the specified type and amount

    `input_dir`: Directory that contains the tables without any homographs
    `output_dir`: Directory where the injected homographs dataset is stored
    `homograph_info_df`: dataframe that contains the homograph information, types, subtypes, files location etc.
    `num_homographs`: number of unique homographs to be inserted overall
    `type`: the type of homographs to be inserted
    `subtype`: the subtype of homographs to be inserted
    `seed`: seed used for the random selection of homographs from the set of valid homographs
    `mode`: if specified, then all types are considered, for each type `num_homographs` are selected

    Returns a list of the selected homographs
    '''
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    if mode=='all_types':
        # Select homographs from each type
        selected_homographs=[]
        selected_homographs.extend(select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type='traditional', subtype=None, seed=seed))
        selected_homographs.extend(select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type='symbolic', subtype='code', seed=seed))
        selected_homographs.extend(select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type='symbolic', subtype='numeric', seed=seed))
        selected_homographs.extend(select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type='null_equivalent', subtype=None, seed=seed))
    else:
        selected_homographs=select_homographs_of_specified_type(homograph_df=homograph_info_df, num_homographs=num_homographs, type=type, subtype=subtype, seed=seed)

    valid_tuples_df=homograph_info_df[homograph_info_df['value'].isin(selected_homographs)]
    files_to_modify=set(valid_tuples_df['filename'].unique())
    for filename in tqdm(os.listdir(input_dir)):
        if filename in files_to_modify:
            # Perform insertion
            orig_file_df=pd.read_csv(input_dir+filename, keep_default_na=False)
            tuples_arr=[]
            for idx, row in valid_tuples_df[valid_tuples_df['filename']==filename].iterrows():
                tuples_arr.append(row['contents_row'])
            new_file_df = pd.concat([orig_file_df, pd.DataFrame(tuples_arr, columns=orig_file_df.columns)])
            new_file_df.to_csv(output_dir+filename, index=False)
        else:
            # Copy the file
            shutil.copyfile(input_dir+filename, output_dir+filename)

    return selected_homographs

def generate_null_equivalent_tuples(input_dir, homograph_info_df, num_meanings=10, num_values=10, seed=0):
    '''
    Returns a dataframe with with content tuples to be used for null-equivalent values

    `input_dir`: input directory containing the raw csv files to be used for the generation
    `homograph_info_df`: dataframe that contains the homograph information, types, subtypes, files location etc., used to ensure no null-equivalent value is found in that dataframe
    `num_meanings`: number of meanings of the generated null_equivalent values (i.e., number of unique column_names selected)
    `num_values`: number of unique null-equivalent values to be formed
    
    Returns a dataframe with the tuples for the null equivalent values
    '''
    random.seed(seed)

    # The string values of the null equivalent values
    null_equivalent_vals = ['null_equivalent_val_'+str(i) for i in range(num_values)]

    # Dictionary mapping each column_name to list of filenames containing them
    column_name_to_filenames_dict={}
    for filename in os.listdir(input_dir):
        columns=pd.read_csv(input_dir+filename, keep_default_na=False, nrows=0).columns.to_list()
        for col in columns:
            if col not in column_name_to_filenames_dict:
                column_name_to_filenames_dict[col]=[filename]
            else:
                column_name_to_filenames_dict[col].append(filename)
    
        # Select `num_meanings` (filename, column_name) pairs for each injected_null_equivalent_val
    null_val_to_filename_column_name_pairs={}
    for val in null_equivalent_vals:
        selected_column_names=random.choices(list(column_name_to_filenames_dict.keys()), k=num_meanings)
        filename_column_name_tuples=[(random.choice(column_name_to_filenames_dict[column_name]), column_name) for column_name in selected_column_names]
        null_val_to_filename_column_name_pairs[val]=filename_column_name_tuples
    
    # Construct the tuples and populate the tuples_dict
    tuples_dict={'value': [], 'filename': [], 'column_name' : [], 'type': [], 'subtype': [], "contents_row": []}
    for val in tqdm(null_equivalent_vals):
        for pair in null_val_to_filename_column_name_pairs[val]:
            filename, column_name=pair[0], pair[1]
            df=pd.read_csv(input_dir+filename, keep_default_na=False)

            # Sample a row and modify the value from the selected column_name
            row = df.sample(n=1, random_state=seed)
            row[column_name]=val

            # Update the tuples_dict
            tuples_dict['value'].append(val)
            tuples_dict['filename'].append(filename)
            tuples_dict['column_name'].append(column_name)
            tuples_dict['type'].append('null_equivalent')
            tuples_dict['subtype'].append(None)
            tuples_dict['contents_row'].append(row.values[0])
    
    return pd.DataFrame.from_dict(tuples_dict)

# Homograph Injection and Dataset Construction

In [69]:
raw_input_dir='../DATA/synthetic_benchmark_large3/'
output_dir='datasets/synthetic_benchmark_large3/'

with open('column_name_to_homograph_type.json') as f:
    column_name_to_homograph_type = json.load(f)
val_to_col_name_dict = get_val_to_col_name_dict(input_dir=raw_input_dir)
homograph_info_df=get_homographs_and_clean_dataset(output_dir=output_dir, input_dir=raw_input_dir, column_name_to_homograph_type=column_name_to_homograph_type, val_to_col_name_dict=val_to_col_name_dict)

100%|██████████| 192/192 [00:12<00:00, 15.16it/s]


In [70]:
print("The input dataset contains:", homograph_info_df['value'].nunique(), "unique homograph values of which:",
      homograph_info_df[homograph_info_df['type']=='traditional']['value'].nunique(), 'are traditional', 
      homograph_info_df[homograph_info_df['subtype']=='code']['value'].nunique(), 'are symbolic(code)',
      homograph_info_df[homograph_info_df['subtype']=='numeric']['value'].nunique(), 'are symbolic(numeric)')

The input dataset contains: 2083 unique homograph values of which: 852 are traditional 387 are symbolic(code) 1178 are symbolic(numeric)


In [71]:
# Append the null_tuples_df into the homograph_info_df
null_tuples_df = generate_null_equivalent_tuples(input_dir=raw_input_dir, homograph_info_df=homograph_info_df, num_meanings=10, num_values=100)
homograph_info_df=pd.concat([homograph_info_df, null_tuples_df])
homograph_info_df.to_pickle(output_dir+'homograph_info_df.pickle')
homograph_info_df

100%|██████████| 100/100 [00:02<00:00, 43.83it/s]


,value,filename,column_name,type,subtype,contents_row
0,1,row_id_animal_name_scientific_name_1.csv,row_id,symbolic,numeric,"[1, Southern tamandua, Tamandua tetradactyla]"
1,2,row_id_animal_name_scientific_name_1.csv,row_id,symbolic,numeric,"[2, Buffalo, american, Bison bison]"
2,3,row_id_animal_name_scientific_name_1.csv,row_id,symbolic,numeric,"[3, Thomson's gazelle, Gazella thompsonii]"
3,4,row_id_animal_name_scientific_name_1.csv,row_id,symbolic,numeric,"[4, Wallaby, whip-tailed, Macropus parryi]"
4,5,row_id_animal_name_scientific_name_1.csv,row_id,symbolic,numeric,"[5, Brazilian tapir, Tapirus terrestris]"
...,...,...,...,...,...,...
995,null_equivalent_val_99,grocery_country_9.csv,product_grocery,null_equivalent,None,"[null_equivalent_val_99, China]"
996,null_equivalent_val_99,street_name_street_number_postal_code_6.csv,street_name,null_equivalent,None,"[null_equivalent_val_99, 2, 34-103]"
997,null_equivalent_val_99,plant_name_scientific_name_family_12.csv,plant_scientific_name,null_equivalent,None,"[Perfumeballs, null_equivalent_val_99, Asterac..."
998,null_equivalent_val_99,employee_id_full_name_company_name_5.csv,full_name,null_equivalent,None,"[45-2296222, null_equivalent_val_99, Feedmix]"


In [81]:
homograph_info_df[homograph_info_df['subtype']=='numeric']['value'].unique()[1000:1100]

array(['2205', '1156', '1888', '2720', '1123', '928', '1500', '3250',
       '2621', '1200', '1375', '3850', '4520', '1253', '88', '968',
       '1109', '1300', '3543', '225', '2630', '228', '900', '5102',
       '4751', '1267', '4225', '760', '207', '2625', '9500', '430',
       '1095', '1114', '1555', '2529', '1002', '6527', '9205', '2715',
       '5022', '914', '960', '750', '2334', '5600', '6000', '1410', '240',
       '4500', '1950', '330', '2434', '626', '4900', '924', '1602',
       '4000', '1236', '550', '6201', '1240', '2559', '1213', '4212',
       '5509', '525', '4513', '745', '2612', '5355', '2340', '3903',
       '545', '2430', '5020', '1000', '5507', '3412', '2000', '2021',
       '1682', '1260', '6415', '6070', '4313', '2302', '3700', '1808',
       '4307', '1110', '3600', '3707', '5815', '1252', '1296', '1679',
       '1174', '2284', '1304'], dtype=object)

## Injected Datasets Construction

In [101]:
homograph_info_df=pd.read_pickle(output_dir+'homograph_info_df.pickle')

num_homographs=100
traditional_homs = construct_homograph_injected_dataset(input_dir=output_dir+'no_homographs/', output_dir=output_dir+'homographs_traditional_'+str(num_homographs)+'/', homograph_info_df=homograph_info_df, num_homographs=num_homographs, type='traditional', seed=0)
symbolic_code_homs = construct_homograph_injected_dataset(input_dir=output_dir+'no_homographs/', output_dir=output_dir+'homographs_symbolic_code_'+str(num_homographs)+'/', homograph_info_df=homograph_info_df, num_homographs=num_homographs, type='symbolic', subtype='code', seed=0)
symbolic_numeric_homs = construct_homograph_injected_dataset(input_dir=output_dir+'no_homographs/', output_dir=output_dir+'homographs_symbolic_numeric_'+str(num_homographs)+'/', homograph_info_df=homograph_info_df, num_homographs=num_homographs, type='symbolic', subtype='numeric', seed=0)
null_equivalent_homs = construct_homograph_injected_dataset(input_dir=output_dir+'no_homographs/', output_dir=output_dir+'homographs_null_equivalent_'+str(num_homographs)+'/', homograph_info_df=homograph_info_df, num_homographs=num_homographs, type='null_equivalent', seed=0)
all_types_homs = construct_homograph_injected_dataset(input_dir=output_dir+'no_homographs/', output_dir=output_dir+'homographs_all_'+str(num_homographs)+'/', homograph_info_df=homograph_info_df, num_homographs=num_homographs, type=None, seed=0, mode='all_types')

homs_dict={
    "homographs_traditional": list(traditional_homs),
    "homographs_symbolic_code": list(symbolic_code_homs),
    "homographs_symbolic_numeric": list(symbolic_numeric_homs),
    "homographs_null_equivalent": list(null_equivalent_homs),
    "homographs_all": list(all_types_homs)
}
with open(output_dir+'selected_homographs.json', 'w') as f:
    json.dump(homs_dict, f, indent=4)

100%|██████████| 192/192 [00:02<00:00, 65.97it/s]


# Dataset Statistics

In [5]:
import pickle
with open('../graph_construction/combined_graphs_output/synthetic_benchmark_large3/no_homographs/bipartite/bipartite.graph', 'rb') as f:
    G=pickle.load(f)
print(G.number_of_nodes(), G.number_of_edges())

129413 211901


# Testing

In [56]:
np.random.seed(0)
np.random.choice(homograph_info_df[(homograph_info_df['type']=='traditional')]['value'].unique(), size=10, replace=False)

array(['Colorado', 'Duff', 'Elan', 'Garvey', 'Elmira', 'Berkeley', 'Ram',
       'Charity', 'California', 'Crossfire'], dtype=object)

In [30]:
homograph_info_df[homograph_info_df['type']=='symbolic']['value'].unique()

array(['ID', 'NE', 'GT', 'AR', 'CO', 'MA', 'CA', 'DE', 'TL', 'MN', 'AL',
       'SD', 'PA', 'AZ', 'TN', 'CT', 'SC', 'IL', 'GA', 'MD', 'ME',
       'Colorado', 'Elan', 'ES', 'Crossfire', 'Ram', 'California',
       'Jimmy', nan], dtype=object)

In [37]:
val_to_col_name_dict[7]

{'airport_elevation', 'cost', 'row_id'}

In [50]:
homs_tmp=get_homographs_and_clean_dataset(output_dir='../DATA/tmp/', input_dir='../DATA/testing/', column_name_to_homograph_type=column_name_to_homograph_type, val_to_col_name_dict=val_to_col_name_dict)
homs_tmp

100%|██████████| 12/12 [00:00<00:00, 12.49it/s]


,value,filename,column_name,type,subtype,contents_row
0,0,row_id_airport_code_continent_code_airport_ele...,airport_code,symbolic,code,"[0, Usina Mandu Airport, 1597]"
1,10,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[CCK, Cocos (Keeling) Islands Airport, 10]"
2,0,row_id_airport_code_continent_code_airport_ele...,airport_code,symbolic,code,"[0, Venâncio Aires Airport, 226]"
3,100,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[NDA, Bandanaira Airport, 100]"
4,10,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[ARD, Mali Airport, 10]"
...,...,...,...,...,...,...
1139,900,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[ZZV, Zanesville Municipal Airport, 900]"
1140,13,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[ARO, Arboletes Airport, 13]"
1141,626,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[LNN, Willoughby Lost Nation Municipal Airport..."
1142,9,row_id_airport_code_continent_code_airport_ele...,airport_elevation,symbolic,numeric,"[FID, Elizabeth Field, 9]"


In [55]:
homs_tmp['value'].unique()

array(['0', '10', '100', '9', '13', '200', '90', '15', '650', '98', '164',
       '545', '42', '760', 'RVR', '325', '740', 'DNA', '88', '940', 'SIS',
       '3500', 'MKZ', '80', '300', '1500', 'TSX', '207', '1', '6000',
       '928', 'MPV', '120', '228', 'LHS', '57', '600', '745', 'SRX',
       '626', 'LUV', 'GTO', '430', '323', 'SLX', '2500', '750', '924',
       'GLI', '550', '62', '911', 'SSR', 'LOL', '960', '900', '9000',
       '525', '240', '500', '944', '330', 'MKS', '645', 'STS', '1000',
       'GTI', '968', 'APV', '914', '929', 'DBS', 'GLC', 'LSS', 'RSX'],
      dtype=object)